## Instalar e habilitar Julia
Baseado [neste template](https://colab.research.google.com/github/ageron/julia_notebooks/blob/master/Julia_Colab_Notebook_Template.ipynb).

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
# JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES="IJulia"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

# Avaliações condicionais em Julia
18 de janeiro de 2020, Pedro
### Conteúdo
  1. Revisão de comparações
  2. Variáveis e expressões *booleanas*
  3. Operadores "e", "ou", e "não" (`&&`, `||` e `!`)
  4. `parse()` e `readline()`
  5. `if - elseif - else`
  
### Fontes
  * [Julia documentation - Conditional Evaluation](https://docs.julialang.org/en/v1/manual/control-flow/#man-conditional-evaluation)
  * [KALICHARAN, Noel. Julia - Bit by Bit. Programming for Begginers. Cap. 3. 2021.](https://doi.org/10.1007/978-3-030-73936-2)

## Variáveis booleanas
Nas últimas semanas, vimos um pouco sobre comparações entre números. Aprendemos a avaliar expressões como:

In [8]:
n = 8  # Guardando 8 em uma variável chamada n.
n < 9

true

In [4]:
n > 123

false

In [5]:
n == 8

true

In [6]:
n < -Inf

false

In [7]:
n != NaN

true

Todas essas expressões *retornam* somente dois possíveis valores, `true` ou `false`. Expressões e variáveis que funcionam desse jeito são chamadas **condicionais**, ou ***booleanas***, e surgem naturalmente em diversos contextos ao se programar. Elas podem ser usadas para controlar se determinadas partes do código vão ser executadas ou não, como veremos adiante.

Os resultados das expressões booleanas acima poderiam ser então salvos em uma variável:

In [10]:
variavel_booleana = (n > Inf)  # Os parênteses são opcionais, só pra visualizar melhor.

false

Enquanto o valor dessa variável é `false`, seu tipo, quando perguntado à Julia, é booleano:

In [17]:
println(variavel_booleana)
println(typeof(variavel_booleana))

false
Bool


Podemos então usar o valor dessa expressão para decidir se vamos rodar um pedaço de código ou não:

In [22]:
variavel_booleana_falsa = false

if variavel_booleana
    println("O valor da variável booleana é true.")
end

if variavel_booleana_falsa
    println("A Julia não vai escrever essa frase.")
end

No lugar das variáveis no exemplo acima, podemos, na verdade usar qualquer expressão booleana, que retorne `true` ou `false`:

In [28]:
if (3 + 5) / 4 == 2
    println("A álgebra ainda funciona.")
end

if (3 + 5) / 4 != 2
    println("Algo tá errado.")
end

A álgebra ainda funciona.
τ vale 6.283185307179586


Pra não repetir código, poderíamos escrever o código de cima usando `!`, o operador booleano de negação. Ele é usado para inverter o valor da expressão booleana em que é aplicado, por exemplo:

In [30]:
o_céu_é_verde = false

if !o_céu_é_verde
    println("O céu não é verde!")
end

O céu não é verde!


Embora ainda não seja o melhor jeito, o outro exemplo poderia então ser escrito assim:

In [31]:
expressão_booleana = (3 + 5) / 4 == 2

if expressão_booleana
    println("A álgebra ainda funciona.")
    # Pode ter quanto código aqui quanto você quiser,
    # o código abaixo é só qualquer blablablá pra exemplificar.
    b = π
    tau = b * 2
    b -= 23
    b = b * sin(tau) ÷ 3
end

if !expressão_booleana
    println("Algo tá errado.")
end

A álgebra ainda funciona.
τ vale 6.283185307179586


Note também que o segundo `if` é pra executar sempre que o primeiro não executar, e existe uma palavra-chave em Julia justamente pra simplificar esses casos, a `else`. Um `else` cria um bloco de código que roda se o `if` não rodar. O jeito mais conciso de se escrever o exemplo seria então algo como:

In [32]:
if (3 + 5) / 4 == 2
    println("A álgebra ainda funciona.")
else
    println("Algo tá errado.")
end

A álgebra ainda funciona.


Podemos combinar duas expressões booleanas com os operadores "e" e "ou", que são escritos em Julia como `&&` e `||`, respectivamente. `a && b` será `true` somente se `a` for `true` *e* `b` for `true`, enquanto `a || b` será verdadeira mesmo se um dos dois for falso.

In [51]:
n = 5
(n == 3) || (n == 5)

true

In [52]:
(n > 6) && (n > 1)

false

In [107]:
n > 6 || !(n > 1)

false

In [54]:
n > 40 || n > 10

false

Usar expressões condicionais é o primeiro jeito que vemos de se controlar a execução de um programa, determinando quais linhas vão ser executadas e *se* elas vão ser executadas. Controlar o programa dessa forma fica mais interessante se adicionarmos uma camada de interatividade.

In [38]:
mensagem_do_usuário = readline()

if mensagem_do_usuário == "oi"
    println("Oi!")
else
    println("Só respondo se disser 'oi'.")
end

stdin>  oi


Oi!


Caso queiramos receber um número, podemos transformar um pedaço de texto (chamado *string* em programação) com a função `parse`. Essa função recebe dois argumentos: o tipo ao qual queremos converter e a *string* a ser convertida. Por exemplo: 

In [40]:
numero = parse(Int, "543123")
println(typeof(numero))

Int64


In [44]:
meu_texto = "1.4321"
println(typeof(meu_texto))
meu_número = parse(Float64, meu_texto)
println(typeof(meu_número))

String
Float64


A gente poderia usar isso em um programa que recebe a nota de um aluno e diz se ele foi aprovado, com nota maior que 5:

In [46]:
nota = parse(Float64, readline())

if nota >= 5
    println("O aluno foi aprovado!")
else
    println("O aluno ainda não foi aprovado :(")
end

stdin>  77


O aluno foi aprovado!


Mas pode ser que queiramos também testar se ele está de recuperação, com nota entre 3 e 5. Se fizermos assim, pode ser que dê errado (tente com um número maior que 5):

In [46]:
# Esse código tá errado!
nota = parse(Float64, readline())

if nota >= 5
    println("O aluno foi aprovado! :D")
end
if nota >= 3
    println("O aluno tá de recuperação :/")
else
    println("O aluno foi reprovado :(")
end

stdin>  77


O aluno foi aprovado!


Daria pra usarmos o que aprendemos antes, combinado com o operador `&&`:

In [49]:
# Código feio, mas pelo menos funciona.
nota = parse(Float64, readline())

if nota >= 5
    println("O aluno foi aprovado! :D")
end
if (nota >= 3) && (nota < 5)
    println("O aluno tá de recuperação :/")
end
if nota < 3
    println("O aluno foi reprovado :(")
end

stdin>  6


O aluno foi aprovado! :D


Ou ainda, se formos mais engenhosos, podemos escrever menos código:

In [57]:
# Ideia interessante... Mas ainda não.
nota = parse(Float64, readline())

if nota >= 5
    println("O aluno foi aprovado! :D")
else
    if nota >= 3
        println("O aluno tá de recuperação :/")
    else
        println("O aluno foi reprovado :(")
    end
end

stdin>  2


O aluno foi reprovado :(


Como vários `if`s encadeados desse jeito são úteis para não precisar escrever um monte de expressões, como no penúltimo exemplo, mas são chatos de se ler e interpretar, criou-se a palavra-chave `elseif`, que faz exatamente a mesma coisa do exemplo anterior, mas de forma mais sucinta:

In [57]:
# Agora sim!
nota = parse(Float64, readline())

if nota >= 5
    println("O aluno foi aprovado! :D")
elseif nota >= 3
    println("O aluno tá de recuperação :/")
else
    println("O aluno foi reprovado :(")
end

stdin>  2


O aluno foi reprovado :(


Dá pra gente entender o `elseif` como um `if` que só roda se o de cima não rodar. Podemos usar vários desses ao mesmo tempo, pra testar diversas condições de forma legível. Imagine a bagunça que seria reescrever o exemplo abaixo usando `if`s e `else`s encadeados:

In [63]:
nota = parse(Float64, readline())

if nota >= 9
    println("O aluno tirou pelo menos 9!")
elseif nota >= 8
    println("A nota do aluno está entre 8 e 9.")
elseif nota >= 7
    println("A nota do aluno está entre 7 e 8.")
elseif nota >= 6
    println("A nota do aluno está entre 6 e 7.")
elseif nota >= 5
    println("A nota do aluno está entre 5 e 6.")
elseif nota >= 4
    println("A nota do aluno está entre 4 e 5.")
elseif nota >= 3
    println("A nota do aluno está entre 3 e 4.")
elseif nota >= 2
    println("A nota do aluno está entre 2 e 3.")
elseif nota >= 1
    println("A nota do aluno está entre 1 e 2.")
elseif nota >= 0
    println("A nota do aluno está entre 0 e 1.")
else
    println("O aluno tirou uma nota negativa!")
end

stdin>  5.5


A nota do aluno está entre 5 e 6.


Reiterando, o `elseif` é um `if` que só roda se *nenhum* `if` acima rodar.

Para encerrar, podemos brincar de adivinhar um número secreto que o programa escolheu. Rode a célula abaixo pra selecionar um número secreto.

In [98]:
# Só vamos entender essa linha melhor em uma próxima reunião. Por ora, basta sa-
# ber que ela guarda na variável segredo um número aleatório entre 1 e 100.
segredo = rand(1:101)
println("Escolhi um número!")

Escolhi um número!


Rode a célula abaixo pra dar um chute, você pode tentar várias vezes.

In [105]:
chute = parse(Int, readline())
if chute == segredo
    println("Parabéns, você acertou!")
elseif chute > segredo
    println("Tente um menor!")
else
    println("Tente um maior!")
end

stdin>  98


Parabéns, você acertou!


Rode a célula abaixo pra revelar qual número foi escolhido.

In [106]:
println("O número era ", segredo)

O número era 98


Por fim, um exercício interessante, caso tenha interesse, é tentar inverter os papéis: escrever uma célula que tenta adivinhar um número que você escolheu e vai mudando o chute dela cada vez que você a roda, até acertar.